In [1]:
import glob
import pandas as pd
from tensorflow import keras
import numpy as np
import os 
from sklearn.model_selection import StratifiedKFold
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, BatchNormalization, concatenate, Input, ConvLSTM2D, Reshape, Conv3D, Flatten, LSTM, GRU, Dense,Dropout, Add
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalMaxPool1D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Sequential, load_model
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import re 

import nltk # for stopwords 
from nltk.corpus import stopwords
import gensim # for Word2Vec embeddings 
from gensim.models import KeyedVectors
from sentencepiece import SentencePieceTrainer,SentencePieceProcessor
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
import googletrans 
from googletrans import Translator



In [2]:
train = pd.read_csv('./storage/writer/train.csv') 
test = pd.read_csv('./storage/writer/test_x.csv') 
ss = pd.read_csv('./storage/writer/sample_submission.csv') 

In [3]:
x_train = train['text'] 
y_train = train['author'] 
x_test = test['text']

In [4]:
x_train = np.asarray(x_train) 
y_train = np.asarray(y_train) 
x_test = np.asarray(x_test)

In [5]:
x_train.shape, y_train.shape, x_test.shape 

((54879,), (54879,), (19617,))

In [8]:
backtrans_xtrain = [] 
translator = Translator()
for i in range(30):   
    translated = translator.translate(x_train[i], dest = 'fr') 
    backtranslated = translator.translate(translated.text, dest = 'en') 
    backtrans_xtrain.append(backtranslated.text)

In [9]:
backtrans_xtrain = np.asarray(backtrans_xtrain) 

x_train = np.concatenate([x_train, backtrans_xtrain]) 
y_train = np.concatenate([y_train, y_train[:30]]) 

In [10]:
x_train.shape, y_train.shape

((54909,), (54909,))

In [11]:
vocab_size = 20000 
maxlength = 256 
embedding_dim = 20 

tokenizer = Tokenizer(num_words = vocab_size, lower = True)
tokenizer.fit_on_texts(x_train)

In [12]:
train_sequences = tokenizer.texts_to_sequences(x_train)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=maxlength)

test_sequences = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=maxlength)


In [13]:
def build_model(): 
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=maxlength),
        tf.keras.layers.GlobalAveragePooling1D(),
        # tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model 

In [14]:
# conduct KFold Ensemble  
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 777) 
for idx, (train_idx,val_idx) in enumerate(kfold.split(train_padded, y_train)):
    
    print("... Validating on Fold {} ...".format(idx+1))
    
    # split data into train and validation sets 
    cur_x_train, cur_x_val = train_padded[train_idx], train_padded[val_idx] 
    cur_y_train, cur_y_val = y_train[train_idx], y_train[val_idx] 
    
    # build model, define callbacks and train  
    model_path = './storage/writer_trainfiles2/kfold' + str(idx+1) + '/epoch_{epoch:03d}_val_{val_loss:.3f}.h5'
    model = build_model() 
    learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 1, verbose = 1, factor = 0.5)
    checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 10) 
    history = model.fit(cur_x_train,
                        cur_y_train,
                        validation_data = (cur_x_val,cur_y_val),
                        shuffle = True,
                        batch_size = 256, 
                        epochs = 250,
                        verbose = 1,
                        callbacks = [learning_rate_reduction, checkpoint, early_stopping]) 

... Validating on Fold 1 ...
Train on 43927 samples, validate on 10982 samples
Epoch 1/250
43008/43927 [============================>.] - ETA: 0s - loss: 1.5696 - accuracy: 0.2727
Epoch 00001: val_loss improved from inf to 1.55549, saving model to ./storage/writer_trainfiles2/kfold1/epoch_001_val_1.555.h5
43927/43927 [==============================] - 3s 74us/sample - loss: 1.5694 - accuracy: 0.2725 - val_loss: 1.5555 - val_accuracy: 0.2750
Epoch 2/250
43776/43927 [============================>.] - ETA: 0s - loss: 1.5463 - accuracy: 0.2783
Epoch 00002: val_loss improved from 1.55549 to 1.53573, saving model to ./storage/writer_trainfiles2/kfold1/epoch_002_val_1.536.h5
43927/43927 [==============================] - 2s 47us/sample - loss: 1.5462 - accuracy: 0.2782 - val_loss: 1.5357 - val_accuracy: 0.2833
Epoch 3/250
42752/43927 [============================>.] - ETA: 0s - loss: 1.5208 - accuracy: 0.3166
Epoch 00003: val_loss improved from 1.53573 to 1.50547, saving model to ./storage/wr

### Make predictions

In [16]:
model1 = load_model('./storage/writer_trainfiles2/kfold1/epoch_093_val_0.606.h5') 
model2 = load_model('./storage/writer_trainfiles2/kfold2/epoch_090_val_0.580.h5') 
model3 = load_model('./storage/writer_trainfiles2/kfold3/epoch_096_val_0.591.h5')
model4 = load_model('./storage/writer_trainfiles2/kfold4/epoch_094_val_0.599.h5')
model5 = load_model('./storage/writer_trainfiles2/kfold5/epoch_086_val_0.601.h5')

In [17]:
pred1 = model1.predict_proba(test_padded)
pred2 = model2.predict_proba(test_padded)
pred3 = model3.predict_proba(test_padded)
pred4 = model4.predict_proba(test_padded)
pred5 = model5.predict_proba(test_padded)

In [19]:
pred_avg = (pred1 + pred2 + pred3 + pred4 + pred5)/5.0 
ss[['0','1','2','3','4']] = pred_avg
ss.head() 


,index,0,1,2,3,4
0,0,1.465800e-04,8.781241e-01,0.117624,4.070309e-03,0.000036
1,1,2.121087e-01,5.559049e-01,0.043457,3.319281e-02,0.155336
2,2,9.841048e-01,8.670364e-03,0.002677,3.614671e-05,0.004512
3,3,5.452764e-07,5.446355e-15,0.999574,5.604425e-16,0.000425
4,4,9.909714e-01,5.796687e-03,0.000245,2.569548e-03,0.000417


In [ ]:
ss.to_csv('./storage/')